In [1]:
# Import necessary modules
import pandas as pd
import numpy as np
import glob
import netCDF4 as nc
from datetime import datetime, timedelta
import pandas as pd
import xarray as xr
import os
import time
from functions.fluxfile_functions import *
from functions.background_functions import *
from functions.obspack_identification_col import *
from functions.obspack_identification_url import *
import tqdm
from multiprocessing import Pool
from functools import partial
import argparse

/home/dkivits/.conda/envs/cte-hr-env/lib/python3.11/site-packages/icoscp/cpauth/exceptions.py:42: FutureWarning: 
The ICOS Carbon Portal python library (>=0.1.20) requires user authentication for external users.
Only credentials used for password sign-in at https://cpauth.icos-cp.eu/ can be used for authentication.
Internal users (ICOS CP Jupyter Notebook services) are exempt.
For the authentication module documentation, follow this link: https://icos-carbon-portal.github.io/pylib/modules/#authentication
To suppress this message we refer to the documentation here: https://icos-carbon-portal.github.io/pylib/faq/#how-do-i-suppress-warnings
  warnings.warn(warning, category=FutureWarning)


In [73]:
ds = '/projects/0/ctdas/PARIS/DATA/obspacks/BASE_SS/pseudo_co2_rgl_tower-insitu_160_allvalid-90magl_2021-07-01-00:00-2021-08-01-00:00.nc'

ds = xr.open_dataset(ds)

variable = 'time'
print(ds.sel(time=slice(datetime(2021, 7, 4, 18, 0), datetime(2021,7,5,15,0)))[variable].values)
print(len(ds.sel(time=slice(datetime(2021, 7, 4, 18, 0), datetime(2021,7,5,15,0)))[variable].values))

variable = 'mixed'
print(ds.sel(time=slice(datetime(2021, 7, 4, 18, 0), datetime(2021,7,5,15,0)))[variable].values)
print(len(ds.sel(time=slice(datetime(2021, 7, 4, 18, 0), datetime(2021,7,5,15,0)))[variable].values))




['2021-07-04T20:00:00.000000000' '2021-07-04T21:00:00.000000000'
 '2021-07-04T22:00:00.000000000' '2021-07-04T23:00:00.000000000'
 '2021-07-05T00:00:00.000000000' '2021-07-05T01:00:00.000000000'
 '2021-07-05T02:00:00.000000000' '2021-07-05T03:00:00.000000000'
 '2021-07-05T04:00:00.000000000' '2021-07-05T05:00:00.000000000'
 '2021-07-05T06:00:00.000000000' '2021-07-05T07:00:00.000000000'
 '2021-07-05T08:00:00.000000000' '2021-07-05T09:00:00.000000000'
 '2021-07-05T10:00:00.000000000' '2021-07-05T11:00:00.000000000'
 '2021-07-05T12:00:00.000000000' '2021-07-05T13:00:00.000000000'
 '2021-07-05T14:00:00.000000000' '2021-07-05T15:00:00.000000000']
20
[            nan             nan             nan             nan
             nan             nan             nan             nan
             nan             nan             nan             nan
             nan             nan             nan -3.01766944e-06
 -1.60601076e-06 -2.88435923e-06 -3.28931542e-06 -2.45271445e-06]
20


In [54]:
outpath = '/projects/0/ctdas/PARIS/DATA/obs/obspack_co2_466_GVeu_20230913/nc/co2_rgl_tower-insitu_160_allvalid-90magl.nc'
ds = xr.open_dataset(outpath)

variable = 'value'
print(ds.sel(time=slice(datetime(2021, 7, 3, 15, 0), datetime(2021,7,4,20,0)))[variable].values)
print(len(ds.sel(time=slice(datetime(2021, 7, 3, 15, 0), datetime(2021,7,4,20,30)))[variable].values))
print(ds.sel(time=slice(datetime(2021, 7, 3, 15, 0), datetime(2021,7,4,20,0)))[variable].values)
print(len(ds.sel(time=slice(datetime(2021, 7, 3, 15, 0), datetime(2021,7,4,20,30)))[variable].values))


[0.00041132 0.00041053 0.00040893 0.00041187 0.00041391 0.00041654
 0.00041875 0.00042075 0.00042034 0.00042125 0.0004238  0.0004306
 0.0004294  0.00042668 0.00042584 0.00043138 0.00042724 0.00042035
 0.0004176  0.00041414 0.00041377 0.00041228]
23
[0.00041132 0.00041053 0.00040893 0.00041187 0.00041391 0.00041654
 0.00041875 0.00042075 0.00042034 0.00042125 0.0004238  0.0004306
 0.0004294  0.00042668 0.00042584 0.00043138 0.00042724 0.00042035
 0.0004176  0.00041414 0.00041377 0.00041228]
23


In [19]:
outpath = '/projects/0/ctdas/PARIS/DATA/obspacks/BASE_SS/'
stationcode = 'rgl'

# Combine all partial netCDF ObsPack files and delete the intermediate files
partial_files= sorted(glob.glob(outpath + '/pseudo_co2_' + stationcode + '_*2021*.nc'))
print(partial_files)
filelist = [xr.open_dataset(file) for file in partial_files]
complete_filestr = outpath + 'pseudo_co2_' + stationcode + '_surface-insitu_25_allvalid.nc'
if os.path.exists(complete_filestr):
    os.remove(complete_filestr)

# Merge datasets along the 'time' dimension
#combined_dataset = xr.merge(filelist, join='outer')
combined_dataset = xr.concat(filelist, dim='time')


print(combined_dataset)
print(combined_dataset['time'].where(combined_dataset['mixed'][:] == None).dropna('time').values)
#for file in partial_files:
#    os.remove(file)    

['/projects/0/ctdas/PARIS/DATA/obspacks/BASE_SS/pseudo_co2_rgl_tower-insitu_160_allvalid-90magl_2021-07-01-00:00-2021-08-01-00:00.nc']
<xarray.Dataset>
Dimensions:                    (time: 579, calendar_components: 6,
                                dim_concerns: 6)
Coordinates:
  * time                       (time) datetime64[ns] 2021-07-01 ... 2021-07-3...
Dimensions without coordinates: calendar_components, dim_concerns
Data variables: (12/31)
    start_time                 (time) datetime64[ns] 2021-07-01 ... 2021-07-3...
    datetime                   (time) |S20 b'2021-07-01T00:00:00.' ... b'2021...
    time_decimal               (time) float64 2.021e+03 2.021e+03 ... 2.022e+03
    time_components            (time, calendar_components) float32 2.021e+03 ...
    solartime_components       (time, calendar_components) float32 2.021e+03 ...
    value                      (time) float32 0.0004065 0.0004061 ... 0.0004173
    ...                         ...
    flux_ocean_exchange_prio

In [42]:
from multiprocessing import Pool

foo = {1: []}

def f(x):
    return x

def main():
    pool = Pool()
    foo[1] = pool.map(f, range(100))
    pool.close()
    pool.join()
    print(foo)

if __name__ == '__main__':
    main()

{1: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]}


In [44]:
from multiprocessing import Pool, current_process
def my_function(arg1,arg2):
    rank = f'running on process: {current_process()._identity[0]}'
    print(rank, arg1,arg2)


arg1 = 'arg1'
arg2 = 'arg2'
with Pool(processes=5) as pool:
    pool.starmap(my_function, [(arg1,arg2),(arg1,arg2)])
    pool.close()
    pool.join()

print(current_process().name)
print("should only run once all processes are finished")

TypeError: my_function() missing 1 required positional argument: 'arg2'